In [124]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn import metrics

In [125]:
path=r"C:\Users\hkshi\OneDrive\Desktop\Soil types"
classes=["Black Soil","Cinder Soil","Laterite Soil","Peat Soil","Yellow Soil"]
images=[]
label=[]
for soil in classes:
    npath=os.path.join(path,soil)
    index=classes.index(soil)
    for imgs in os.listdir(npath):
        nnpath=os.path.join(npath,imgs)
        img=cv.imread(nnpath)
        img.resize(128,128,3)
        images.append(img)
        label.append(index)
images=np.array(images)
label=np.array(label)

In [126]:
model=tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=(128,128,3)),
    tf.keras.layers.Conv2D(8,(3,3),strides=(1,1),padding="same",activation="relu"),
    tf.keras.layers.Conv2D(16,(3,3),strides=(1,1),padding="same",activation="relu"),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(16,(3,3),strides=(1,1),padding="same",activation="relu"),
    tf.keras.layers.Conv2D(32,(3,3),strides=(1,1),padding="same",activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5,activation="sigmoid"),
])
model.compile(loss="sparse_categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_15 (Bat  (None, 128, 128, 3)      12        
 chNormalization)                                                
                                                                 
 conv2d_60 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_61 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 dropout_31 (Dropout)        (None, 64, 64, 16)        0         
                                                                 
 conv2d_62 (Conv2D)          (None, 64, 64, 16)      

In [130]:
x_train,x_test,y_train,y_test=train_test_split(images,label,test_size=0.25)
aug=ImageDataGenerator(channel_shift_range=0.4,rotation_range=0.4)
aug.fit(x_train)
model.fit_generator(aug.flow(x_train,y_train,batch_size=3),epochs=10,validation_data=(x_test,y_test))

Epoch 1/10


C:\Users\hkshi\AppData\Local\Temp\ipykernel_2436\1707744581.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(aug.flow(x_train,y_train,batch_size=3),epochs=10,validation_data=(x_test,y_test))


39/39 [==============================] - 4s 96ms/step - loss: 0.3251 - accuracy: 0.8783 - val_loss: 0.1851 - val_accuracy: 0.9487
Epoch 2/10
39/39 [==============================] - 4s 105ms/step - loss: 0.4549 - accuracy: 0.8783 - val_loss: 0.3168 - val_accuracy: 0.8462
Epoch 3/10
39/39 [==============================] - 6s 146ms/step - loss: 0.2527 - accuracy: 0.9217 - val_loss: 0.1084 - val_accuracy: 0.9744
Epoch 4/10
39/39 [==============================] - 7s 184ms/step - loss: 0.0658 - accuracy: 0.9913 - val_loss: 0.1065 - val_accuracy: 0.9487
Epoch 5/10
39/39 [==============================] - 8s 195ms/step - loss: 0.0435 - accuracy: 0.9913 - val_loss: 0.1148 - val_accuracy: 0.9487
Epoch 6/10
39/39 [==============================] - 4s 108ms/step - loss: 0.0876 - accuracy: 0.9652 - val_loss: 0.0972 - val_accuracy: 0.9487
Epoch 7/10
39/39 [==============================] - 4s 93ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.1078 - val_accuracy: 0.9487
Epoch 8/10
39/39 [=

In [131]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 27ms/step - loss: 0.1265 - accuracy: 0.9487


[0.12652626633644104, 0.9487179517745972]

In [132]:
ypred=model.predict(x_train)
newpred=[]
ypred=np.array(ypred)
ypred.shape

4/4 [==============================] - 0s 76ms/step


(115, 5)

In [133]:
for pred in ypred:
    pred1=np.argmax(pred)
    newpred.append(pred1)

In [134]:
newpred=np.array(newpred)
newpred.shape

(115,)

In [140]:
model.save(r"C:\Users\hkshi\OneDrive\Desktop\hackmolsoilclassification.h5")

In [135]:
from sklearn.metrics import f1_score,precision_score,recall_score
f1_score(y_train,newpred,average="micro")
precision_score(y_train,newpred,average="micro")
recall_score(y_train,newpred,average="micro")

1.0

In [136]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,newpred)

array([[25,  0,  0,  0,  0],
       [ 0, 21,  0,  0,  0],
       [ 0,  0, 22,  0,  0],
       [ 0,  0,  0, 24,  0],
       [ 0,  0,  0,  0, 23]], dtype=int64)

In [137]:
newpred

array([1, 1, 1, 1, 2, 2, 4, 2, 1, 1, 3, 3, 2, 4, 0, 0, 0, 0, 1, 0, 0, 4,
       1, 4, 3, 4, 0, 0, 3, 4, 3, 2, 0, 0, 2, 3, 2, 1, 0, 3, 4, 2, 0, 2,
       1, 0, 0, 1, 2, 4, 4, 0, 2, 2, 3, 0, 3, 3, 1, 0, 1, 3, 3, 3, 1, 1,
       3, 1, 3, 4, 0, 2, 3, 4, 0, 0, 3, 3, 4, 0, 1, 2, 2, 1, 4, 2, 4, 0,
       2, 4, 2, 4, 1, 2, 2, 3, 4, 4, 3, 4, 1, 0, 4, 0, 3, 4, 4, 3, 2, 0,
       2, 3, 1, 3, 4], dtype=int64)

In [138]:
y_train

array([1, 1, 1, 1, 2, 2, 4, 2, 1, 1, 3, 3, 2, 4, 0, 0, 0, 0, 1, 0, 0, 4,
       1, 4, 3, 4, 0, 0, 3, 4, 3, 2, 0, 0, 2, 3, 2, 1, 0, 3, 4, 2, 0, 2,
       1, 0, 0, 1, 2, 4, 4, 0, 2, 2, 3, 0, 3, 3, 1, 0, 1, 3, 3, 3, 1, 1,
       3, 1, 3, 4, 0, 2, 3, 4, 0, 0, 3, 3, 4, 0, 1, 2, 2, 1, 4, 2, 4, 0,
       2, 4, 2, 4, 1, 2, 2, 3, 4, 4, 3, 4, 1, 0, 4, 0, 3, 4, 4, 3, 2, 0,
       2, 3, 1, 3, 4])